<a href="https://colab.research.google.com/github/Sahil01S/Sentiment_analysis_rnn/blob/main/sentiment_analysis_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving train.tsv to train.tsv
Saving test.tsv to test.tsv
Saving sampleSubmission.csv to sampleSubmission.csv


In [ ]:
import pandas as pd

# Load the training data
train_df = pd.read_csv('train.tsv', sep='\t')
print("Training Data:")
print(train_df.head())

# Load the test data
test_df = pd.read_csv('test.tsv', sep='\t')
print("Test Data:")
print(test_df.head())

# Load the sample submission data
sample_submission_df = pd.read_csv('sampleSubmission.csv')
print("Sample Submission Data:")
print(sample_submission_df.head())


Training Data:
   PhraseId  SentenceId                                             Phrase  \
0         1           1  A series of escapades demonstrating the adage ...   
1         2           1  A series of escapades demonstrating the adage ...   
2         3           1                                           A series   
3         4           1                                                  A   
4         5           1                                             series   

   Sentiment  
0          1  
1          2  
2          2  
3          2  
4          2  
Test Data:
   PhraseId  SentenceId                                             Phrase
0    156061        8545  An intermittently pleasing but mostly routine ...
1    156062        8545  An intermittently pleasing but mostly routine ...
2    156063        8545                                                 An
3    156064        8545  intermittently pleasing but mostly routine effort
4    156065        8545         intermit

In [ ]:
!pip install gensim


In [ ]:
import pandas as pd
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords


# Convert all entries to string type
train_df['Phrase'] = train_df['Phrase'].astype(str)
test_df['Phrase'] = test_df['Phrase'].astype(str)


In [ ]:
# Fill missing values with an empty string
train_df['Phrase'].fillna('', inplace=True)
test_df['Phrase'].fillna('', inplace=True)


In [ ]:


# Preprocessing function
def preprocess_text(text):
    text = remove_stopwords(text)  # Remove stop words
    tokens = simple_preprocess(text)  # Tokenize and clean
    return tokens

# Apply preprocessing
train_df['tokens'] = train_df['Phrase'].apply(preprocess_text)
test_df['tokens'] = test_df['Phrase'].apply(preprocess_text)


In [ ]:
print(train_df[['Phrase', 'tokens']].head())
print(test_df[['Phrase', 'tokens']].head())


                                              Phrase  \
0  A series of escapades demonstrating the adage ...   
1  A series of escapades demonstrating the adage ...   
2                                           A series   
3                                                  A   
4                                             series   

                                              tokens  
0  [series, escapades, demonstrating, adage, good...  
1  [series, escapades, demonstrating, adage, good...  
2                                           [series]  
3                                                 []  
4                                           [series]  
                                              Phrase  \
0  An intermittently pleasing but mostly routine ...   
1  An intermittently pleasing but mostly routine ...   
2                                                 An   
3  intermittently pleasing but mostly routine effort   
4         intermittently pleasing but mostly routine 

In [ ]:
from gensim.models import Word2Vec

# Create a list of tokenized sentences
sentences = train_df['tokens'].tolist() + test_df['tokens'].tolist()

# Train Word2Vec model
word2vec_model = Word2Vec(sentences, vector_size=300, window=5, min_count=1, workers=4)

# Save the model
word2vec_model.save('word2vec.model')


In [ ]:
from gensim.models import Word2Vec

# Load pre-trained Word2Vec model (replace with your file path)
word2vec_model = Word2Vec.load('word2vec.model')


In [ ]:
import numpy as np

# Create embedding matrix
embedding_dim = 300
word_index = {word: i for i, word in enumerate(word2vec_model.wv.index_to_key)}
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))

for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Flatten tokens into a single list for tokenizer
all_tokens = train_df['tokens'].tolist() + test_df['tokens'].tolist()
all_tokens_flat = [word for tokens in all_tokens for word in tokens]

# Initialize tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_tokens_flat)

# Convert text to sequences
X_train = tokenizer.texts_to_sequences(train_df['tokens'])
X_test = tokenizer.texts_to_sequences(test_df['tokens'])

# Pad sequences
max_len = max([len(x) for x in X_train])  # Find the maximum length
X_train_padded = pad_sequences(X_train, maxlen=max_len)
X_test_padded = pad_sequences(X_test, maxlen=max_len)

# Prepare labels
y_train = to_categorical(train_df['Sentiment'])


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(5, activation='softmax'))  # Adjust to match sentiment classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_padded, y_train, epochs=5, batch_size=64, validation_split=0.2)


Epoch 1/5
1951/1951 [==============================] - 237s 119ms/step - loss: 1.1744 - accuracy: 0.5322 - val_loss: 1.1836 - val_accuracy: 0.5223
Epoch 2/5
1951/1951 [==============================] - 226s 116ms/step - loss: 1.1201 - accuracy: 0.5552 - val_loss: 1.1748 - val_accuracy: 0.5305
Epoch 3/5
1951/1951 [==============================] - 226s 116ms/step - loss: 1.0754 - accuracy: 0.5760 - val_loss: 1.1637 - val_accuracy: 0.5341
Epoch 4/5
1951/1951 [==============================] - 248s 127ms/step - loss: 1.0334 - accuracy: 0.5926 - val_loss: 1.1950 - val_accuracy: 0.5301
Epoch 5/5
1951/1951 [==============================] - 225s 115ms/step - loss: 0.9957 - accuracy: 0.6081 - val_loss: 1.1948 - val_accuracy: 0.5263


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_train_padded, y_train)
print(f"Loss: {loss}, Accuracy: {accuracy}")

# Predict sentiments
predictions = model.predict(X_test_padded)
predicted_classes = np.argmax(predictions, axis=1)

# Create a DataFrame for submission
submission_df = pd.DataFrame({
    'PhraseId': test_df['PhraseId'],
    'Sentiment': predicted_classes
})

submission_df.to_csv('submission.csv', index=False)


4877/4877 [==============================] - 148s 30ms/step - loss: 1.0035 - accuracy: 0.6083
Loss: 1.0035053491592407, Accuracy: 0.6083173155784607
2072/2072 [==============================] - 61s 29ms/step


The model is giving descent accuracy. We may have better accuracy if we do more epochs.